# Illustration of mix-SQP solver applied to a small data set, and a large one

## Analysis setup

*Before attempting to run this Julia code, make sure your computer is properly set up to run this code by following the "Quick Start" instructions in the README of the [git repository](https://github.com/stephenslab/mixsqp-paper).*

We begin by loading the Distributions and LowRankApprox Julia packages, as well as some function definitions used in the code chunks below.

In [1]:
using Distributions
using LowRankApprox
include("../code/datasim.jl");
include("../code/likelihood.jl");
include("../code/mixSQP.jl");

Next, initialize the sequence of pseudorandom numbers.

In [2]:
srand(1);

## Generate a small data set

Let's start with a smaller example with 50,000 samples.

In [3]:
x = normtmixdatasim(round(Int,5e4));

## Compute the likelihood matrix

Compute the $n \times k$ likelihood matrix for a mixture of zero-centered normals, with $k = 20$. Note that the rows of the likelihood matrix are normalized by default.

In [4]:
sd = autoselectmixsd(x,nv = 20);
L  = normlikmatrix(x,sd = sd);
size(L)

(50000, 20)

## Fit mixture model using SQP algorithm 

First we run the mix-SQP algorithm once to precompile the function.

In [5]:
out = mixSQP(L,verbose = false);

Observe that only a small number of iterations is needed to converge to the solution of the constrained optimization problem.

In [6]:
k   = size(L,2);
x0  = ones(k)/k;
out = mixSQP(L,x = x0);

Running SQP algorithm with the following settings:
- 50000 x 20 data matrix
- convergence tolerance = 1.00e-08
- zero threshold        = 1.00e-03
- partial SVD tolerance  = 1.00e-08
- partial SVD max. error = 4.18e-07
iter      objective -min(g+1) #nnz #qp #ls
   1 3.03733620e+04 +6.30e-01   20   0   0
   2 2.09533189e+04 +5.80e+04    1  12   1
   3 1.28095047e+04 +2.01e+04    4  60   1
   4 1.11151171e+04 +8.72e+03    4  51   1
   5 1.09368106e+04 +4.16e+03    4  12   1
   6 1.07215837e+04 +2.02e+03    3  22   1
   7 1.05948783e+04 +1.03e+03    3  10   1
   8 1.05175827e+04 +5.10e+02    3  10   1
   9 1.03021156e+04 +2.51e+02    2  22   1
  10 1.01826126e+04 +1.28e+02    3  21   1
  11 1.01288446e+04 +6.49e+01    3  12   1
  12 1.00408173e+04 +3.21e+01    3  10   1
  13 9.89770711e+03 +1.61e+01    3  18   1
  14 9.85096785e+03 +8.03e+00    3   7   1
  15 9.81522511e+03 +3.86e+00    3   6   1
  16 9.77452162e+03 +1.82e+00    3  12   1
  17 9.75253943e+03 +8.31e-01    4  12   1
  18 9.7

## Generate a larger data set

Next, let's see what happens when we use the SQP algorithm to fit a mixture model to a much larger data set.

In [7]:
srand(1);
x = normtmixdatasim(round(Int,1e5));

## Compute the likelihood matrix

As before, we compute the $n \times k$ likelihood matrix for a mixture of zero-centered normals. This time, we use a finer grid of $k = 40$ normal densities.

In [8]:
sd = autoselectmixsd(x,nv = 40);
L  = normlikmatrix(x,sd = sd);
size(L)

(100000, 40)

## Fit mixture model using SQP algorithm 

Even on this much larger data set, only a small number of iterations is needed to compute the solution.

In [9]:
k   = size(L,2);
x0  = ones(k)/k;
out = mixSQP(L,x = x0);

Running SQP algorithm with the following settings:
- 100000 x 40 data matrix
- convergence tolerance = 1.00e-08
- zero threshold        = 1.00e-03
- partial SVD tolerance  = 1.00e-08
- partial SVD max. error = 1.08e-06
iter      objective -min(g+1) #nnz #qp #ls
   1 6.21694207e+04 +6.60e-01   40   0   0
   2 4.83207934e+04 +3.93e-01   40  17   2
   3 3.75596757e+04 +2.25e-01   40  52   2
   4 2.99843228e+04 +1.23e-01   40  51   2
   5 2.13835266e+04 +7.20e+03    3  48   1
   6 1.98795777e+04 +2.53e+03    4  65   1
   7 1.97777586e+04 +1.12e+03    3  11   1
   8 1.97192133e+04 +5.22e+02    3  12   1
   9 1.96312101e+04 +2.54e+02    3  28   1
  10 1.95996264e+04 +1.25e+02    4  12   1
  11 1.95641472e+04 +6.16e+01    4  28   1
  12 1.95418605e+04 +3.09e+01    5  12   1
  13 1.95188571e+04 +1.54e+01    5  28   1
  14 1.95042410e+04 +7.72e+00    4   8   1
  15 1.94888486e+04 +3.80e+00    4  21   1
  16 1.94788433e+04 +1.87e+00    5   6   1
  17 1.94701551e+04 +8.57e-01    5  22   1
  18 1.

With no low-rank approximation (`lowrank = "none"`), the algorithm still converges even when using a very small correction factor `eps = 1e-12`.

In [10]:
out = mixSQP(L,x = x0,lowrank = "none",eps = 1e-12);

Running SQP algorithm with the following settings:
- 100000 x 40 data matrix
- convergence tolerance = 1.00e-08
- zero threshold        = 1.00e-03
- Exact derivative computation (partial QR not used).
iter      objective -min(g+1) #nnz #qp #ls
   1 6.21694226e+04 +6.60e-01   40   0   0
   2 4.35976361e+04 +2.74e+08    2  17   1
   3 2.63754248e+04 +9.42e+07    3 100   1
   4 2.26716550e+04 +4.11e+07    3  33   1
   5 2.22369707e+04 +1.93e+07    3  18   1
   6 2.20822494e+04 +9.86e+06    3  10   1
   7 2.17850756e+04 +4.96e+06    3  43   1
   8 2.15686849e+04 +2.48e+06    2  11   1
   9 2.13181589e+04 +1.28e+06    3  39   1
  10 2.11397369e+04 +6.40e+05    2  11   1
  11 2.08812795e+04 +3.39e+05    3  39   1
  12 2.07941248e+04 +1.75e+05    3  10   1
  13 2.04915356e+04 +8.97e+04    3  37   1
  14 2.03990501e+04 +4.57e+04    3  10   1
  15 2.01797507e+04 +2.27e+04    3  16   1
  16 2.00663424e+04 +1.17e+04    3  10   1
  17 2.00068966e+04 +5.91e+03    3  10   1
  18 1.98269277e+04 +3.06

## Session information

The section gives information about the computing environment used to generate the results contained in this
notebook, including the version of Julia, and the versions of the Julia packages used here. 

In [11]:
Pkg.status("Distributions");
Pkg.status("LowRankApprox");
versioninfo();

 - Distributions                 0.15.0
 - LowRankApprox                 0.1.1
Julia Version 0.6.2
Commit d386e40c17 (2017-12-13 18:08 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin14.5.0)
  CPU: Intel(R) Core(TM) i7-7567U CPU @ 3.50GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Prescott)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, broadwell)
